In [19]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import copy

In [20]:
dataset = pd.read_csv('data.csv')
# dataset

In [21]:
X = dataset.drop('xAttack', axis = 1)
Y = dataset['xAttack']

In [22]:
X = (X - np.mean(X))/np.std(X)
X.shape

(125973, 29)

In [23]:
X = X.iloc[:]
# X_copy = copy.deepcopy(X_train)
# Y_copy = copy.deepcopy(Y_train)
# Y_train
Y=pd.concat([Y,pd.get_dummies(Y,prefix='xAttack')],axis=1).drop(['xAttack'],axis=1)
Y = Y.iloc[:]
# X

In [24]:
class Neural_network(object):
    
    def __init__(self):
        self.input_layer_size = 29
        self.output_layer_size = 29
        self.hidden_layer_size = 14
        self.hidden_layers = 1
        self.weights = {}
        self.dweights = {}
        self.Z = {}
        self.a = {}
        self.Y_hat = np.array([])
        self.delta = {}
        self.epoch = 50
        self.ita = 0.0000001
        
    def initialization(self):
        
        self.weights[1] = np.random.randn(self.input_layer_size,self.hidden_layer_size)*0.01
        self.weights[self.hidden_layers+1] = np.random.randn(self.hidden_layer_size,self.output_layer_size)*0.01
        
        for i in np.arange(2,self.hidden_layers+1):
            self.weights[i] = np.random.randn(self.hidden_layer_size,self.hidden_layer_size)*0.01
            
    
    def softmax(self,a):
        tsum = np.sum(a,axis = 1,keepdims = True)
        soft_a = np.divide(a,tsum)
        return soft_a
    
#     def y_prep(self,Y,k):
#          # Onehot
#         y = np.zeros((len(Y),self.output_layer_size))
#         for i in range(y.shape[0]):
#             y[i][Y[i]] = 1
#         return y
    
    
    def error(self,y):
#         return sum(0.5*np.sum((y-self.Y_hat)*(y-self.Y_hat),axis = 1))/y.shape[0]
        return np.mean(0.5*np.sum((y-self.Y_hat)*(y-self.Y_hat),axis = 1))
    
    def derivative_sigmoid(self,layer):
        w = np.ones(layer.shape)
        return w
    
   
    def deltas(self,y):
        w = self.a[self.hidden_layers+2]
        self.delta[self.hidden_layers+2] = -(y - self.Y_hat)*self.derivative_sigmoid(w)
        self.dweights[self.hidden_layers+1] = np.dot(self.a[self.hidden_layers+1].T , (self.delta[self.hidden_layers+2]))
        
        for i in np.arange(self.hidden_layers+1,1,-1):
            self.delta[i] = ( np.dot(self.delta[i+1], self.weights[i].T))*self.derivative_sigmoid(self.a[i])
            self.dweights[i-1] = np.dot(self.a[i-1].T , (self.delta[i]))  
    
    def sigmoid(self,x):
        return x
    
    
    
    def forward_propagation(self, X):
        self.a[1]=np.array(X)
        

        for i in range(1,self.hidden_layers+2):
            self.Z[i+1] = np.dot(self.a[i],self.weights[i])
            self.a[i+1] = self.sigmoid(self.Z[i+1])
        self.Y_hat = self.softmax(self.a[self.hidden_layers+2])
        
   
    def gradient_descent(self,X_train,Y_train):
        k = 0
        while k<=120000:

            X = X_train[k:k+10000]
            y = Y_train[k:k+10000]
#             print(y)
            for i in range(self.epoch):
                self.forward_propagation(X)
#                 y = self.y_prep(Y,k)
#                 print("Error : ",self.error(y))
                self.deltas(y)
                for j in range(1,self.hidden_layers+2):
                    self.weights[j] = self.weights[j] - self.ita*self.dweights[j]
#             print("Final Error : ",self.error(y))
            k = k + 10000
        
        
    def predict(self,X):
        self.a[1] = np.array(X.T[0:self.input_layer_size].T)
        self.Z[1] = np.dot(self.a[1],self.weights[1])
        self.a[1] = self.sigmoid(self.Z[1])
        return self.a[1]
    
    def accuracy(self,Y):
        predicted = np.argmax(self.Y_hat,axis = 1)
        print(predicted)
        print("Accuracy score : ",100*(accuracy_score(predicted,np.array(Y))))
            

In [25]:
k = Neural_network()
k.initialization()
k.gradient_descent(X,X)

In [26]:
# X_test = X_test[0:1000]
# Y_test = Y_test[0:1000]

finaldata=k.predict(X)
finaldata=pd.DataFrame(finaldata)
df_final = pd.concat([finaldata,dataset['xAttack']],axis=1)

In [27]:
df_final.to_csv("data_final_linear.csv",index=False)